In [1]:
import pickle
import numpy as np

def encrypt(m, ck, r): # change ck to dictionary????
    p,q,g,h = ck
    return (pow(g, int(m), p) * pow(h,r,p)) % p # we make sure that m is of type int

def productModp(prodList, p):
    prod = 1
    for t in prodList:
        prod = (prod * t) % p
    return prod

Com = lambda ck, M, r: encrypt(M, ck, r)

# Function to check if vote code is opened.
# C: vote code
# Uopen: shuffled list of m tuples (votecode, rand)
# Open: shuffled list of m tuples (rank, rand)
# Eopen: opened ranks
# U: m commited votecodes
# E: m commited ranks 
def isOpened(C, Uopen, Open, Eopen, U, E, ck):
    # find randomness of vote code C
    index = -1
    myRand = -1
    for i in range(len(Uopen)):
        if C == Uopen[i][0]:
            index = i
            myRand = Uopen[i][1]
            break
    myU = Com(ck, C, myRand)
    
    # locate myU in original commitment
    origIndex = -1
    for i in range(len(U)):
        if myU == U[i]:
            origIndex = i
            break
    myRank = E[origIndex]
    
    return (myRank in Eopen) 

verify = True

with open('BB', 'rb') as f:
    pub = pickle.load(f)

# 1 #
P = pub['Pub']['P']
ballots = pub['OpenBallots']
m = len(P)
n = pub['Pub']['n']

x = max(n+1, 2*m - 1)
# there are n ballots
if len(ballots) != n: 
    verify = False

tags = set()
vote_codes = set()
# print(tags)
for b in ballots:
    if not verify:
        break
    if b[1][0] in tags:
        verify = False
    tags.add(b[1][0])
    for i in range(m):
        if b[0][i][1] in vote_codes or b[1][2][i] in vote_codes:
            verify = False
        vote_codes.add(b[0][i][1])
        vote_codes.add(b[1][2][i])
        
if len(vote_codes) != 2*n*m: ### needed?
    verify = False
# 2 #
Uopen = pub['Uopen']
(Open, Eopen) = pub['Eopen']
U = [pub['Pub']['Publ'][i][1] for i in range(n)]
E = [pub['Pub']['Publ'][i][2] for i in range(n)]
ck = pub['Pub']['ck']

for b in ballots:
    if not verify:
        break
    for i in range(m):
        if not isOpened(b[0][i][1], Uopen[b[1][0]-1][1-b[1][1]], Open[b[1][0]-1], Eopen[b[1][0]-1], U[b[1][0]-1][1-b[1][1]], E[b[1][0]-1][1-b[1][1]], ck):
            verify = False
        if isOpened(b[1][2][i], Uopen[b[1][0]-1][b[1][1]], Open[b[1][0]-1], Eopen[b[1][0]-1], U[b[1][0]-1][b[1][1]], E[b[1][0]-1][b[1][1]], ck):
            verify = False

# 3 #

# 4 #
verify = ([[Com(ck, opening[0], opening[1]) for opening in x]for x in Open] == np.array(Eopen)).all()
Cordered = [[ballots[i][0][j][1] for j in range(m)] for i in range(n)]
Copen = [[[Uopen[i][a][j][0] for j in range(m)] for a in range(2)] for i in range(n)]
votes = [ballots[i][1] for i in range(n)]
# Order of vote_codes (from the side not voted)
Order = [dict(zip(c, range(m))) for c in Cordered]

for v in votes:
    for j in range(m):
        verify = (Open[v[0]-1][j] == pow(x, Order[v[0]-1][Copen[v[0]-1][1- v[1]][j]], ck[0]))
# 5 #
Etally, Esum, (T,R) = pub["Result"]
verify = ([[productModp(tally, ck[0]) for tally in Etally]] == np.array(Esum)).all()
# Each vote contains exactly m vote_codes which belong to the correct ballot
for v in votes:
    v_codes = set(v[2])
    verify = (len(v_codes) == m)
    for c in Copen[v[0]-1][v[1]]:
        verify = (c in v_codes)
         
print(verify)

True


In [2]:
zip([1,2,3], range(3))